<a href="https://colab.research.google.com/github/seokhee516/kaggle-practice/blob/main/Binary%20classification/Titanic%3A%20Machine%20Learning%20from%20Disaster/Introduction%20to%20Ensembling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, 
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn.svm import SVC
from sklearn.model_selection import KFold

In [3]:
!pip install gcsfs

     |████████████████████████████████| 1.3 MB 7.5 MB/s 
     |████████████████████████████████| 119 kB 74.5 MB/s 
     |████████████████████████████████| 142 kB 72.2 MB/s 
     |████████████████████████████████| 294 kB 72.8 MB/s 


In [4]:
GCS_DS_PATH = "gs://kds-181a4deafa270c5ce4ace71ede22337472ee9dfc903a7326ed8770cb"
train = pd.read_csv(GCS_DS_PATH+'/train.csv')
test = pd.read_csv(GCS_DS_PATH+"/test.csv")
IDtest = test["PassengerId"]

train.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
